In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df_train = pd.read_csv('../input/spaceship-titanic/train.csv')
#df_train = df_train.set_index('PassengerId')
df_train

In [ ]:
df_test = pd.read_csv('../input/spaceship-titanic/test.csv')
#df_test = df_test.set_index('PassengerId')
df_test

In [ ]:
df_train.isnull().sum().sum()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.isnull().sum()*100/len(df_train)

In [ ]:
df_test.isnull().sum()*100/len(df_test)

In [ ]:
df_train['isTrain'] = 'Yes'

In [ ]:
df_test['isTrain'] = 'No'

In [ ]:
tt = pd.concat([df_train, df_test])

In [ ]:
tt

In [ ]:
tt = tt.drop(['Name', 'Transported'], axis = 1)
tt

In [ ]:
tt.isnull().sum()

In [ ]:
tt.dtypes

In [ ]:
cat_cols = list(tt.select_dtypes(include=['category', object]).columns)
cat_cols

In [ ]:
num_cols = list(tt.select_dtypes(exclude=['category', object]).columns)
num_cols

In [ ]:
tt_cat = tt[cat_cols]
tt_cat

In [ ]:
tt_cat.isnull().sum()

In [ ]:
tt_cat.isnull().sum()*100/len(tt_cat)

In [ ]:
tt_cat.isnull().sum().sum()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
icat_tt = pd.DataFrame(imputer.fit_transform(tt_cat), columns = tt_cat.columns)
icat_tt

In [ ]:
icat_tt.isnull().sum().sum()

import category_encoders as ce
encoder = ce.OrdinalEncoder()
encoder.fit(icat_tt)
icat_tt = encoder.transform(icat_tt)
#icat_tt['PassengerId'] = tt_cat['PassengerId'] 
icat_tt

In [ ]:
tt_num = tt[num_cols]
tt_num

In [ ]:
tt_num.isnull().sum()*100/len(tt_cat)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imp = IterativeImputer(verbose=2, max_iter=30, tol=1e-10, imputation_order='roman')

In [ ]:
inum_tt = pd.DataFrame(imp.fit_transform(tt_num), columns = tt_num.columns)

In [ ]:
inum_tt

In [ ]:
imp_tt = pd.concat([icat_tt, inum_tt], axis = 1)

In [ ]:
imp_tt

In [ ]:
train = imp_tt[imp_tt['isTrain'] == 'Yes']
#train = train.set_index('PassengerId')
train = train.drop('isTrain', axis = 1)
train

In [ ]:
df_train['Transported']

In [ ]:
train

In [ ]:
test = imp_tt[imp_tt['isTrain'] == 'No']
test = test.set_index('PassengerId')
test = test.drop('isTrain', axis = 1)
test

In [ ]:
def bool_to_word(bool):
    if bool == True:
        return "Yes"
    else:
        return "No"

In [ ]:
train[['CryoSleep', 'VIP']] = train[['CryoSleep', 'VIP']].applymap(bool_to_word)
test[['CryoSleep', 'VIP']] = test[['CryoSleep', 'VIP']].applymap(bool_to_word)

In [ ]:
train.dtypes

In [ ]:
y = df_train['Transported']
y

In [ ]:
train = pd.concat([train, y], axis = 1)

In [ ]:
train

In [ ]:
train['Transported'] = train['Transported'].map(bool_to_word)

In [ ]:
X = train.drop('Transported', axis = 1)
X

In [ ]:
y = train['Transported']
y

In [ ]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X, y)):
    train.loc[valid_indicies, "kfold"] = fold

In [ ]:
train.to_csv("train_folds.csv", index=False)

In [ ]:
df = pd.read_csv("train_folds.csv")

In [ ]:
df

In [ ]:
useful_features = [c for c in df.columns if c not in ("PassengerId", "Transported", "kfold")]
df_test = df_test[useful_features]

In [ ]:
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import accuracy_score

In [ ]:
cat_col = list(df.select_dtypes(include=['category', object]).columns)
cat_col.remove('PassengerId')
cat_col.remove('Transported')
cat_col

In [ ]:
df.select_dtypes('number').columns

In [ ]:
final_predictions = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.Transported
    yvalid = xvalid.Transported
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    train_dataset = Pool(data=xtrain,
                     label=ytrain,
                     cat_features=cat_col)

    eval_dataset = Pool(data=xvalid,
                    label=yvalid,
                    cat_features=cat_col)
    
    
    # Initialize CatBoostClassifier
    model = CatBoostClassifier()
    # Fit model
    model.fit(train_dataset)
    
    test_preds = model.predict(eval_dataset)
    final_predictions.append(test_preds)
    print("=*=" * 10, fold, accuracy_score(yvalid, test_preds), "=*=" * 10)

In [ ]:
subb = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
subb

In [ ]:
preds = model.predict(test)

In [ ]:
subb.Transported = preds
subb

In [ ]:
subb.Transported = subb.Transported.map(dict(Yes=True, No=False))
subb

In [ ]:
subb.to_csv('subb_cat.csv', index = None)